# Restricted Two Body Problem: Elliptical Orbits Around a Central Mass

This is the general case of a Keperian orbit.<br>
A light body (e.g. a planet) orbits a heavy central body (e.g. the sun).  The orbit is an ellipse with the primary at one focus.

In [1]:
# Library imports
import tensorflow as tf
import rebound
import numpy as np

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory, TimeHistory
from tf_utils import plot_loss_hist, EpochLoss, TimeHistory
from tf_utils import Identity

from orbital_element import OrbitalElementToConfig, ConfigToOrbitalElement
from orbital_element import make_model_elt_to_cfg, make_model_cfg_to_elt

from r2b_data import make_traj_r2b, make_train_r2b, make_datasets_r2b, make_datasets_earth
from r2b import KineticEnergy_R2B, PotentialEnergy_R2B, AngularMomentum_R2B

In [3]:
# Lightweight serialization
# fname = '../data/r2b/r2bc.pickle'
# vartbl = load_vartbl(fname)

### Generate data sets and an example batch

In [4]:
# Generate one example trajectory
a = 1.0
e = 0.0
inc = 0.0
Omega = 0.0
omega = 0.0
f = 0.0
n_years = 2

inputs_traj, outputs_traj = make_traj_r2b(a=a, e=e, inc=inc, Omega=Omega, omega=omega, f=f, n_years=n_years)

In [5]:
inputs_traj.keys()

dict_keys(['t', 'q0', 'v0', 'mu'])

In [6]:
# Inputs for make_train_r2b
n_traj = 10
n_years = 2
a_min = 0.50
a_max = 32.0
e_max = 0.20
inc_max = np.pi/4.0
seed = 42

In [7]:
# Test make_train_r2b
inputs, outputs= make_train_r2b(n_traj=n_traj, n_years=n_years, a_min=a_min, a_max=a_max, 
                                e_max=e_max, inc_max=inc_max, seed=seed)

In [8]:
# Inputs for make_datasets_r2b
n_traj = 100
vt_split = 0.20
n_years = 2
a_min = 0.50
a_max = 32.0
e_max = 0.20
inc_max = np.pi/4.0
seed = 42
batch_size = 64

In [9]:
ds_trn, ds_val, ds_tst = make_datasets_r2b(n_traj=n_traj, vt_split=vt_split, n_years=n_years, a_min=a_min, a_max=a_max, 
                                e_max=e_max, inc_max=inc_max, seed=seed, batch_size=batch_size)

Loaded data from ../data/r2b/1421569704.pickle.


In [10]:
# Create DataSet objects for toy size problem - earth orbits only (a=1, e=0)
ds_earth_trn, ds_earth_val, ds_earth_tst = make_datasets_earth(n_traj=n_traj, vt_split=vt_split, n_years=n_years)

Loaded data from ../data/r2b/2367906283.pickle.


In [31]:
# Example batch
batch_in, batch_out = list(ds_earth_trn.take(1))[0]
print('Input field names: ', list(batch_in.keys()))
print('Output field names:', list(batch_out.keys()))

t = batch_in['t']
q0 = batch_in['q0']
v0 = batch_in['v0']
mu = batch_in['mu']

q = batch_out['q']
v = batch_out['v']
a = batch_out['a']
q0_rec = batch_out['q0_rec']
v0_rec = batch_out['v0_rec']
H = batch_out['H']
L = batch_out['L']

print(f'Example batch sizes:')
print(f't  = {t.shape}')
print(f'q0 = {q0.shape}')
print(f'v0 = {v0.shape}')
print(f'mu = {mu.shape}')

print(f'q  = {q.shape}')
print(f'v  = {v.shape}')
print(f'a  = {a.shape}')
# print(f'q0_rec = {q0_rec.shape}')
# print(f'v0_rec = {v0_rec.shape}')
print(f'H  = {H.shape}')
print(f'L  = {L.shape}')

Input field names:  ['t', 'q0', 'v0', 'mu']
Output field names: ['q', 'v', 'a', 'q0_rec', 'v0_rec', 'T', 'U', 'H', 'L']
Example batch sizes:
t  = (64, 731)
q0 = (64, 3)
v0 = (64, 3)
mu = (64,)
q  = (64, 731, 3)
v  = (64, 731, 3)
a  = (64, 731, 3)
H  = (64, 731)
L  = (64, 731, 3)


In [32]:
traj_size = 731

tf.debugging.assert_shapes(
    shapes = {
    # Inputs
    t: (batch_size, traj_size),
    q0: (batch_size, 3),
    v0: (batch_size, 3),
    mu: (batch_size,),
    # Outputs
    q: (batch_size, traj_size, 3),
    v: (batch_size, traj_size, 3),
    a: (batch_size, traj_size, 3),
    q0_rec: (batch_size, 3),
    v0_rec: (batch_size, 3),
    H: (batch_size, traj_size),
    L: (batch_size, traj_size, 3),
    })

**Call layers with physics computations**

In [33]:
T = KineticEnergy_R2B()(v)
T.shape

TensorShape([64, 731])

In [34]:
U = PotentialEnergy_R2B()([q, mu])
U.shape

TensorShape([64, 731])

In [35]:
L = AngularMomentum_R2B()([q, v])
L.shape

TensorShape([64, 731, 3])

**Conversion of initial configuration to orbital elements**

In [36]:
qx = q0[:,0]
qy = q0[:,1]
qz = q0[:,2]
vx = v0[:,0]
vy = v0[:,1]
vz = v0[:,2]
inputs_cart = (qx, qy, qz, vx, vy, vz, mu)

In [37]:
elts = ConfigToOrbitalElement()(inputs_cart)
a0, e0, inc0, Omega0, omega0, f0, M0, N0 = elts

In [38]:
# Review shapes
print(f'Example batch sizes:')
print(f'qx   = {qx.shape}')
print(f'vx   = {vx.shape}')
print(f'mu   = {mu.shape}')
print(f'a0   = {a0.shape}')

Example batch sizes:
qx   = (64,)
vx   = (64,)
mu   = (64,)
a0   = (64,)


### Mathematical Model
**Compute position as a function of time from initial orbital elements**

In [89]:
def make_position_model_r2b_math(traj_size = 731):
    """
    Compute orbit positions for the restricted two body problem from 
    the initial orbital elements with a deterministic mathematical model.
    Factory function that returns a functional model.
    """
    # Create input layers 
    t = keras.Input(shape=(traj_size,), name='t')
    q0 = keras.Input(shape=(3,), name='q0')
    v0 = keras.Input(shape=(3,), name='v0')
    
    # Wrap these up into one tuple of inputs for the model
    inputs = (t, q0, v0)
    
    # The gravitational constant; give this shape (1,1) for compatibility with RepeatVector
    # The numerical value mu0 is close to 4 pi^2; see rebound documentation for exact value
    mu0 = tf.constant([[39.476924896240234]])

    # Tuple of inputs for the model converting from configuration to orbital elements
    inputs_c2e = (q0, v0, mu0)

    # Model mapping cartesian coordinates to orbital elements
    model_c2e = make_model_cfg_to_elt()
    
    # Extract the orbital elements of the initial conditions
    a0, e0, inc0, Omega0, omega0, f0, M0, N0 = model_c2e(inputs_c2e)

    # Name the outputs of the orbital elements model for legibility in model summary
    # a0 = Identity(name='a0')(a0)
    # e0 = Identity(name='e0')(e0)
    # inc0 = Identity(name='inc0')(inc0)
    # Omega0 = Identity(name='Omega0')(Omega0)
    # omega0 = Identity(name='omega0')(omega0)
    # f0 = Identity(name='f0')(f0)
    # "Bonus outputs" - mean anomaly and mean motion
    # M0 = Identity(name='M0')(M0)
    # N0 = Identity(name='N0')(N0)

    # Check shapes of initial orbital elements
    batch_size = t.shape[0]
    tf.debugging.assert_shapes(shapes={
        a0: (batch_size, 1),
        e0: (batch_size, 1),
        inc0: (batch_size, 1),
        Omega0: (batch_size, 1),
        omega0: (batch_size, 1),
        mu0: (batch_size, 1),
    }, message='make_position_model_r2b_math / initial orbital elements')
    
    # Reshape t to (batch_size, traj_size, 1)
    t_vec = keras.layers.Reshape(target_shape=(traj_size, 1), name='t_vec')(t)
    
    # Repeat the constant orbital elements to be vectors of shape (batch_size, traj_size)
    a = keras.layers.RepeatVector(n=traj_size, name='a')(a0)
    e = keras.layers.RepeatVector(n=traj_size, name='e')(e0)
    inc = keras.layers.RepeatVector(n=traj_size, name='inc')(inc0)
    Omega = keras.layers.RepeatVector(n=traj_size, name='Omega')(Omega0)
    omega = keras.layers.RepeatVector(n=traj_size, name='omega')(omega0)
    mu = keras.layers.RepeatVector(n=traj_size, name='mu_vec')(mu0)

    # Throwaway - set the true anomaly f to initial f0 plus t
    # This is wrong, but the code should run
    f_vec = keras.layers.RepeatVector(n=traj_size, name='f')(f0)
    f = f_vec + t_vec
    
#     # Check shapes of orbital element calculations
#     tf.debugging.assert_shapes(shapes={
#         a: (batch_size, traj_size, 1),
#         e: (batch_size, traj_size, 1),
#         inc: (batch_size, traj_size, 1),
#         Omega: (batch_size, traj_size, 1),
#         omega: (batch_size, traj_size, 1),
#         f: (batch_size, traj_size, 1),
#         mu: (batch_size, traj_size, 1),
#     }, message='make_position_model_r2b_math / orbital element calcs')

    # Wrap orbital elements into one tuple of inputs for layer converting to cartesian coordinates
    inputs_e2c = (a, e, inc, Omega, omega, f, mu,)
    
    # Convert from orbital elements to cartesian
    qx, qy, qz, vx, vy, vz = OrbitalElementToConfig(name='orbital_element_to_config')(inputs_e2c)
    
    # Wrap up the outputs
    outputs = (qx, qy, qz, vx, vy, vz)
    
#     # Check shapes
#     tf.debugging.assert_shapes(shapes={
#         qx: (batch_size, traj_size, 1),
#         qy: (batch_size, traj_size, 1),
#         qz: (batch_size, traj_size, 1),
#         vx: (batch_size, traj_size, 1),
#         vy: (batch_size, traj_size, 1),
#         vz: (batch_size, traj_size, 1),
#     }, message='make_position_model_r2b_math / outputs')
    
    # Wrap this into a model
    model = keras.Model(inputs=inputs, outputs=outputs, name='model_r2b_math')
    return model

In [90]:
position_model_math = make_position_model_r2b_math(traj_size=traj_size)

In [91]:
print(f't     = {t.shape}')
print(f'a0    = {a0.shape}')
print(f'e0    = {e0.shape}')
print(f'inc0  = {inc0.shape}')
print(f'Omega0= {Omega0.shape}')
print(f'omega0= {omega0.shape}')
print(f'f0    = {f0.shape}')

t     = (64, 731)
a0    = (64,)
e0    = (64,)
inc0  = (64,)
Omega0= (64,)
omega0= (64,)
f0    = (64,)


In [92]:
qx, qy, qz, vx, vy, vz = position_model_math([t, q0, v0])
print(f'qx = {qx.shape}')
print(f'qy = {qy.shape}')
print(f'qz = {qz.shape}')

qx = (64, 731, 1)
qy = (64, 731, 1)
qz = (64, 731, 1)


In [93]:
vx[0,0:5].numpy()

array([[-5.551481 ],
       [-5.5432205],
       [-5.5349197],
       [-5.5265765],
       [-5.518193 ]], dtype=float32)

In [94]:
# keras.utils.plot_model(position_model_math, '../model_plots/r2b/position_model_math.png')

In [95]:
# position_model_math.summary()

**Motion Model: Compute v and a from q using automatic differentiation**<br>
Factory function that accepts any position model<br>
Instantiated here from mathematical position model

In [132]:
class Motion_R2B(keras.Model):
    """Motion for restricted two body problem generated from a position calculation model."""

    def __init__(self, position_model, **kwargs):
        super(Motion_R2B, self).__init__(**kwargs)
        self.position_model = position_model

    def call(self, inputs):
        """
        Compute full orbits for the restricted two body problem.
        Computes positions using the passed position_layer, 
        then uses automatic differentiation for velocity v and acceleration a.
        INPUTS:
            t: the times to report the orbit; shape (batch_size, traj_size)
            q0: the initial position; shape (batch_size, 3)
            v0: the initial velocity; shape (batch_size, 3)
        OUTPUTS:
            q: the position at time t; shape (batch_size, traj_size, 3)
            v: the velocity at time t; shape (batch_size, traj_size, 3)
            a: the acceleration at time t; shape (batch_size, traj_size, 3)
        """
        # Unpack the inputs
        t, q0, v0 = inputs

        # Get the trajectory size and target shape of t
        traj_size = t.shape[1]
        target_shape = (traj_size, 1)

        # Reshape t to have shape (batch_size, traj_size, 1)
        t = keras.layers.Reshape(target_shape=target_shape, name='t')(t)

        # Check shapes after resizing operation; can accept t of shape EITHER 
        # (batch_size, traj_size) or (batch_size, traj_size, 1)
        batch_size = t.shape[0]
        tf.debugging.assert_shapes(shapes={
            t: (batch_size, traj_size, 1),
        }, message='Motion_R2B.call / inputs')
    
        # Evaluation of the position is under the scope of two gradient tapes
        # These are for velocity and acceleration
        with tf.GradientTape(persistent=True) as gt2:
            gt2.watch(t)

            with tf.GradientTape(persistent=True) as gt1:
                gt1.watch(t)       
        
                # Get the position using the input position layer
                position_inputs = (t, q0, v0)
                # The velocity from the position model assumes the orbital elements are not changing
                # Here we only want to take the position output and do a full automatic differentiation
                qx, qy, qz, vx_, vy_, vz_ = self.position_model(position_inputs)
                q = keras.layers.concatenate(inputs=[qx, qy, qz], axis=2, name='q')

                tf.debugging.assert_shapes(shapes={
                    qx: (batch_size, traj_size, 1),
                    qy: (batch_size, traj_size, 1),
                    qz: (batch_size, traj_size, 1),
                    vx_: (batch_size, traj_size, 1),
                    vy_: (batch_size, traj_size, 1),
                    vz_: (batch_size, traj_size, 1),
                }, message='Motion_R2B / outputs of position model')
                
            # Compute the velocity v = dq/dt with gt1
            vx = gt1.gradient(qx, t)
            vy = gt1.gradient(qy, t)
            vz = gt1.gradient(qz, t)
            v = keras.layers.concatenate(inputs=[vx, vy, vz], axis=2, name='v')
            del gt1
            
            tf.debugging.assert_shapes(shapes={
                vx: (batch_size, traj_size, 1),
                vy: (batch_size, traj_size, 1),
                vz: (batch_size, traj_size, 1),
            }, message='Motion_R2B / velocity by automatic differentation')

        # Compute the acceleration a = d2q/dt2 = dv/dt with gt2
        ax = gt2.gradient(vx, t)
        ay = gt2.gradient(vy, t)
        az = gt2.gradient(vz, t)
        # a = keras.layers.concatenate(inputs=[ax, ay, az], name='a')
        a = ax
        del gt2
        
        # Check shapes
#         tf.debugging.assert_shapes(shapes={
#             q: (batch_size, traj_size, 3),
#             v: (batch_size, traj_size, 3),
#             a: (batch_size, traj_size, 3),
#         }, message='Motion_R2B.call / outputs')

        return q, v, a

In [133]:
motion_model_math = Motion_R2B(position_model=position_model_math, name='motion_model')

In [134]:
# motion_model_math.summary()

In [135]:
q, v, a = motion_model_math([t, q0, v0])
print('shape of motion_model outputs:')
print(f'q: {q.shape}')
print(f'v: {v.shape}')
# print(f'acc: {a.shape}')

shape of motion_model outputs:
q: (64, 731, 3)
v: (64, 731, 3)


In [137]:
type(a)

NoneType

In [71]:
v.numpy()[0,0:5,:]

array([[-8.7614697e-01, -4.7325712e-01, -3.1138441e-04],
       [-8.7486190e-01, -4.7565076e-01, -3.1178427e-04],
       [-8.7357020e-01, -4.7804099e-01, -3.1218180e-04],
       [-8.7227190e-01, -4.8042759e-01, -3.1257700e-04],
       [-8.7096715e-01, -4.8281080e-01, -3.1296990e-04]], dtype=float32)